In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import sys
import csv
from batchmake import Batcher
import astwalker
import ast
import tokenize
from io import StringIO

In [2]:
pad_token, pad_id = "§PAD§", 0
oov_token, oov_id = "§OOV§", 1
indent_token = "§<indent>§"
dedent_token = "§<dedent>§"
number_token = "§NUM§"

In [3]:
datas= 'data_samples/'
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
inv_map = {v:k for k, v in word_to_id.items() }
# batch_data = datas+'output.txt.part0'
batch_data = datas+'preprocess.part0'
with open(batch_data, 'rb') as f:
    picked_data = pickle.load(f)
    
print(len(picked_data))
train_proportion = 0.5
valid_proportion = 0.2

481


In [4]:
subdirectories = [os.path.join(datas,o) for o in os.listdir(datas) if os.path.isdir(os.path.join(datas,o))]

In [5]:
python_files = [(directory, [y for x in os.walk(directory) for y in iglob(os.path.join(x[0], '*.py'))])
                for directory in subdirectories]

In [6]:
train_split = int(len(python_files) * train_proportion)
valid_split = train_split + int(len(python_files) * valid_proportion)
train_files = []
valid_files = []
test_files = []

In [7]:
for project in python_files[:train_split]:
    train_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[train_split:valid_split]:
    valid_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[valid_split:]:
    test_files.extend([f[len(datas):] for f in project[1]])

In [8]:
def write_to_file(fname, lst):
    with open(os.path.join(datas, fname), "w") as write_file:
        for f in lst:
            print(f, file=write_file)

In [9]:
write_to_file("train_files.txt", train_files)
write_to_file("valid_files.txt", valid_files)
write_to_file("test_files.txt", test_files)

### 데이터셋을 train, valid, test셋으로 만든다.

### 파일 읽기

In [10]:
python_files = [os.path.join(datas, f) for f in train_files]
mapping = (lambda x: x) if word_to_id is None else (lambda x: word_to_id.get(x, oov_id))
def get_source_tree(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        fstr = f.read()
    fstr = fstr.replace('\r\n', '\n').replace('\r', '\n')
    if not fstr.endswith('\n'):
        fstr += '\n'
    return fstr, ast.parse(fstr, filename=filename)

In [11]:
def preprocess(tokentype, tokenval):
    if tokentype == tokenize.NUMBER:
        return number_token

    elif tokentype == tokenize.INDENT:
        return indent_token

    elif tokentype == tokenize.DEDENT:
        return dedent_token

    return tokenval

In [12]:
data = []
definition_positions = []
identifier_usage = []
gen_def_positions = True
error_data = 0
data_file_name = []
for filename in python_files:
    try:
        source, tree = get_source_tree(filename)
        tokens = tokenize.generate_tokens(StringIO(source).readline)
        data.append([(mapping(preprocess(tokenType, tokenVal)), start) for tokenType, tokenVal, start, _, _
                     in tokens
                     if tokenType != tokenize.COMMENT and
                     not tokenVal.startswith("'''") and
                     not tokenVal.startswith('"""') and 
                     (tokenType == tokenize.DEDENT or tokenVal != "")])
        data_file_name.append(filename)
        if gen_def_positions:
            walker = astwalker.ASTWalker()
            walker.walk(tree)
            definition_positions.append(walker.definition_positions)
            identifier_usage.append(walker.name_usage)
    except:
        error_data+=1
        continue
        print("Error when tokenizing %s: %s" % (filename, sys.exc_info()[:3]))

### 전체 파일 ( python_files ) 중에서 에러난거 뺀데이터 (data) 

In [13]:
len(python_files),error_data,len(data), len(definition_positions)

(534, 55, 479, 479)

### 데이터 비교

In [14]:
d , dd=get_source_tree(data_file_name[0])
print(d)

from __future__ import unicode_literals
import re
import ast
try:
    from setuptools import setup
except ImportError:
    from distutils.core import setup

def function2835():
    with open('zhihu/__init__.py', 'rb') as var2990:
        var4009 = re.search('__version__ = (.*)', var2990.read().decode('utf-8')).group(1)
        if (var4009 is None):
            raise RuntimeError('Cannot find version information')
        return str(ast.literal_eval(var4009))
with open('README.rst', 'rb') as var2178:
    var2523 = var2178.read().decode('utf-8')
var4384 = ['zhihu']
var3646 = function2835()
setup(name='zhihu-py3', version=var3646, keywords=['zhihu', 'network', 'spider', 'html'], description='Zhihu UNOFFICIAL API library in python3, with help of bs4, lxml, requests and html2text.', long_description=var2523, author='7sDream', author_email='didislover@gmail.com', license='MIT', url='https://github.com/7sDream/zhihu-py3', download_url='https://github.com/7sDream/zhihu-py3', install_requires=[

In [15]:
token_to_str = []
for tokend_data,position in data[0]:
     token_to_str.append(inv_map[tokend_data])
print(' '.join(token_to_str))

from __future__ import unicode_literals 
 import re 
 import §OOV§ 
 try : 
 §<indent>§ from §OOV§ import setup 
 §<dedent>§ except ImportError : 
 §<indent>§ from §OOV§ . §OOV§ import setup 
 
 §<dedent>§ def function2835 ( ) : 
 §<indent>§ with open ( §OOV§ , 'rb' ) as var2990 : 
 §<indent>§ var4009 = re . search ( §OOV§ , var2990 . read ( ) . decode ( 'utf-8' ) ) . group ( §NUM§ ) 
 if ( var4009 is None ) : 
 §<indent>§ raise RuntimeError ( §OOV§ ) 
 §<dedent>§ return str ( §OOV§ . §OOV§ ( var4009 ) ) 
 §<dedent>§ §<dedent>§ with open ( §OOV§ , 'rb' ) as var2178 : 
 §<indent>§ var2523 = var2178 . read ( ) . decode ( 'utf-8' ) 
 §<dedent>§ var4384 = [ §OOV§ ] 
 var3646 = function2835 ( ) 
 setup ( name = 'zhihu-py3' , version = var3646 , §OOV§ = [ §OOV§ , §OOV§ , §OOV§ , 'html' ] , description = §OOV§ , §OOV§ = var2523 , author = '7sDream' , §OOV§ = §OOV§ , §OOV§ = §OOV§ , url = §OOV§ , §OOV§ = §OOV§ , §OOV§ = [ §OOV§ , §OOV§ , §OOV§ ] , §OOV§ = { §OOV§ : [ §OOV§ ] , } , packages = v

### 원본데이터와 토큰데이터

In [16]:
type_max_rands = {
    "var": 4750, "function": 2900, "Class": 440, "attribute": 2400, "arg": 2400
}

identifier_types = [key for key in type_max_rands]
def_positions = [[[t[1] for t in fp if t[0] == k] for k in identifier_types] for fp in definition_positions]
# ex) def_positions[101] #function, var, class, attribute, arg

In [43]:
inputs = []
for line_data in data[:30]:
    line_data_list = []
    for token in line_data:
        line_data_list.append(token[0])
        if token[0] == 2:
            inputs.append(line_data_list)
            line_data_list=[]
            
#             print(token[0])
        

In [52]:
for row in inputs:
    print(row)

[38, 455, 19, 3138, 2]
[19, 63, 2]
[19, 1, 2]
[49, 8, 2]
[9, 38, 1, 19, 5719, 2]
[10, 55, 399, 8, 2]
[9, 38, 1, 5, 1, 19, 5719, 2]
[2]
[10, 17, 6866, 3, 4, 8, 2]
[9, 175, 147, 3, 1, 6, 935, 4, 102, 2282, 8, 2]
[9, 1879, 7, 63, 5, 332, 3, 1, 6, 2282, 5, 360, 3, 4, 5, 540, 3, 189, 4, 4, 5, 185, 3, 12, 4, 2]
[16, 3, 1879, 45, 21, 4, 8, 2]
[9, 90, 337, 3, 1, 4, 2]
[10, 18, 71, 3, 1, 5, 1, 3, 1879, 4, 4, 2]
[10, 10, 175, 147, 3, 1, 6, 935, 4, 102, 4405, 8, 2]
[9, 1972, 7, 4405, 5, 360, 3, 4, 5, 540, 3, 189, 4, 2]
[10, 5794, 7, 13, 1, 14, 2]
[5246, 7, 6866, 3, 4, 2]
[5719, 3, 88, 7, 4228, 6, 172, 7, 5246, 6, 1, 7, 13, 1, 6, 1, 6, 1, 6, 1600, 14, 6, 1059, 7, 1, 6, 1, 7, 1972, 6, 190, 7, 4247, 6, 1, 7, 1, 6, 1, 7, 1, 6, 348, 7, 1, 6, 1, 7, 1, 6, 1, 7, 13, 1, 6, 1, 6, 1, 14, 6, 1, 7, 35, 1, 8, 13, 1, 14, 6, 34, 6, 444, 7, 5794, 6, 1, 7, 13, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 14, 4, 2]
[19, 62, 2]
[19, 42, 2]
[62, 5, 50, 5, 1, 3, 12, 6, 42, 5, 50, 5, 6093, 3, 1038, 4, 4, 2]
[9255, 7, 1

[10, 10, 10, 17, 3731, 3, 11, 6, 1300, 4, 8, 2]
[9, 38, 5, 190, 19, 228, 6, 325, 2]
[2456, 7, 1300, 5, 86, 3, 156, 6, 114, 7, 1, 4, 2]
[16, 3, 2456, 5, 268, 45, 21, 4, 8, 2]
[9, 6739, 7, 2456, 5, 445, 5, 219, 13, 774, 14, 2]
[2174, 7, 2456, 5, 445, 5, 219, 13, 211, 14, 2]
[3929, 7, 2456, 5, 445, 5, 1362, 5, 111, 2]
[1385, 7, 3, 6037, 27, 1300, 5, 219, 5, 5615, 13, 1801, 14, 5, 200, 3, 6258, 6, 4098, 4, 4, 2]
[2273, 7, 1300, 5, 199, 3, 1, 4, 2]
[533, 7, 13, 70, 3, 1052, 5, 98, 3, 1563, 5, 1, 3, 4, 4, 5, 185, 3, 12, 4, 4, 29, 1563, 20, 2273, 14, 2]
[18, 228, 3, 2174, 6, 6739, 6, 3929, 6, 21, 6, 533, 13, 12, 14, 6, 533, 13, 12, 14, 6, 533, 13, 12, 14, 6, 533, 13, 12, 14, 6, 1385, 6, 153, 7, 11, 5, 1138, 4, 2]
[10, 32, 8, 2]
[9, 18, 325, 2]
[2]
[10, 10, 73, 89, 2]
[73, 192, 3, 1, 4, 2]
[17, 2612, 3, 11, 4, 8, 2]
[9, 1, 2]
[7817, 7, 11, 5, 196, 5, 1, 3, 1, 4, 2]
[3071, 7, 7817, 5, 111, 2]
[5440, 7, 3071, 5, 87, 3, 4, 13, 12, 14, 2]
[18, 3, 70, 3, 5440, 4, 16, 5440, 5, 2000, 3, 4, 32, 12, 4,

[10, 1069, 7, 104, 3, 193, 3, 2975, 4, 4, 2]
[1069, 5, 752, 3, 267, 7, 11, 5, 5101, 6, 1, 7, 36, 4, 2]
[8094, 7, 363, 5, 199, 3, 156, 6, 114, 7, 1, 4, 13, 3, 47, 12, 4, 14, 13, 4936, 14, 2]
[29, 4407, 20, 1069, 8, 2]
[9, 4765, 7, 3, 233, 27, 4407, 13, 211, 14, 4, 2]
[7744, 7, 4407, 5, 111, 5, 93, 3, 4, 2]
[2360, 7, 296, 3, 4765, 6, 7744, 6, 153, 7, 11, 5, 349, 4, 2]
[136, 227, 2]
[10, 1121, 13, 724, 14, 7, 8094, 2]
[486, 7, 11, 5, 349, 5, 209, 3, 6484, 6, 57, 7, 1121, 4, 2]
[1828, 7, 486, 5, 148, 3, 4, 13, 476, 14, 13, 12, 14, 2]
[16, 3, 1828, 25, 12, 4, 8, 2]
[9, 18, 2]
[10, 363, 7, 260, 3, 486, 5, 148, 3, 4, 13, 476, 14, 13, 12, 14, 4, 2]
[2]
[10, 10, 73, 89, 2]
[17, 5696, 3, 11, 4, 8, 2]
[9, 1, 2]
[38, 5, 227, 19, 296, 2]
[38, 5, 190, 19, 228, 2]
[38, 5, 320, 19, 341, 2]
[2359, 7, 1, 5, 139, 3, 11, 5, 1854, 4, 2]
[4135, 7, 35, 1694, 8, 12, 6, 312, 8, 11, 5, 2824, 6, 34, 2]
[486, 7, 11, 5, 349, 5, 76, 3, 2359, 4, 2]
[363, 7, 260, 3, 486, 5, 397, 4, 2]
[105, 36, 8, 2]
[9, 3861, 7, 363

[160, 5, 843, 3, 4, 2]
[242, 5, 467, 3, 4, 2]
[18, 13, 14, 2]
[2]
[10, 10, 17, 6376, 3, 11, 6, 287, 6, 1331, 4, 8, 2]
[9, 29, 265, 20, 287, 8, 2]
[9, 11, 5, 191, 5, 46, 3, 3, 265, 13, 12, 14, 6, 30, 4, 4, 2]
[16, 3, 11, 5, 2535, 115, 40, 3, 265, 13, 12, 14, 4, 4, 8, 2]
[9, 11, 5, 2535, 7, 40, 3, 265, 13, 12, 14, 4, 2]
[10, 10, 1322, 7, 12, 2]
[29, 265, 20, 287, 8, 2]
[9, 11, 5, 163, 5, 46, 3, 691, 3, 265, 13, 12, 14, 6, 1322, 6, 3, 11, 5, 2535, 27, 12, 4, 6, 12, 4, 4, 2]
[1322, 66, 12, 2]
[10, 11, 5, 2263, 7, 3, 1331, 27, 12, 4, 2]
[11, 5, 1056, 7, 3, 11, 5, 2535, 27, 12, 4, 2]
[11, 5, 132, 5, 3403, 3, 11, 5, 2263, 6, 11, 5, 1056, 4, 2]
[11, 5, 3910, 3, 287, 4, 2]
[2]
[10, 17, 3910, 3, 11, 6, 287, 4, 8, 2]
[9, 49, 8, 2]
[9, 1123, 7, 12, 2]
[29, 265, 20, 287, 8, 2]
[9, 932, 7, 30, 2]
[29, 2168, 20, 100, 3, 12, 6, 3, 3, 11, 5, 2535, 47, 40, 3, 265, 13, 12, 14, 4, 4, 27, 12, 4, 6, 12, 4, 8, 2]
[9, 932, 7, 3, 932, 27, 197, 4, 2]
[10, 11, 5, 132, 5, 140, 3, 1123, 6, 12, 6, 3, 3, 265, 13, 12

In [55]:
with open('inputs.csv', 'w',newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile)
    for row_line in inputs:
        spamwriter.writerow(row_line)

### target은 input+1 의 인덱스로 하고, sequence는 실행시간에 정함.